<a href="https://colab.research.google.com/github/ali-siddiquii/Multi-Document-Query-Assistant-with-OpenAI-and-ChromaDB/blob/main/rag_with_python_flask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
To run this project, create a .env file in the project root directory with the following variables:

---------------------------------------------
API_KEY=your_api_key_here
NGROK_AUTH_TOKEN=your_ngrok_auth_token_here
---------------------------------------------



The .env file will be loaded automatically

"""


In [ ]:

%pip install llama-index-agent-openai
%pip install llama-index-embeddings-openai
%pip install llama-index-llms-openai
%pip install chromadb
%pip install docx2txt
%pip install python-dotenv

%pip install flask
%pip install flask-ngrok

%pip install pyngrok


%pip install llama-index
%pip install python-docx
%pip install pdfminer.six

from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok



from llama_index.core import (
    VectorStoreIndex,
    SimpleKeywordTableIndex,
    SimpleDirectoryReader,
)

!pip install pyngrok
!pip install flask




from flask import Flask, jsonify, make_response, request
from flask_ngrok import run_with_ngrok



import os
import threading

from flask import Flask
from pyngrok import ngrok

from flask import Flask, request, jsonify
from llama_index.core import SummaryIndex
from llama_index.core.schema import IndexNode
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.llms.openai import OpenAI
from llama_index.core.callbacks import CallbackManager

from llama_index.agent.openai import OpenAIAgent
from llama_index.core import load_index_from_storage, StorageContext
from llama_index.core.node_parser import SentenceSplitter
import os

from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
import openai

import chromadb
chroma_client = chromadb.Client()

from dotenv import load_dotenv
from pathlib import Path
from werkzeug.utils import secure_filename
from docx import Document
import os
from pdfminer.high_level import extract_text

load_dotenv()

OPEN_API_KEY = os.getenv('API_KEY')
openai.api_key = os.getenv('API_KEY')
#openai.api_key = os.environ["API_KEY"]
Settings.llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")


ngrok_auth_token = os.getenv('NGROK_AUTH_TOKEN')
!ngrok config add-authtoken $ngrok_auth_token


app = Flask(__name__)
run_with_ngrok(app)

node_parser = SentenceSplitter()
uploaded_docs = []


try:
    collection = chroma_client.create_collection(name="my_collection")
except:
    collection = chroma_client.get_collection(name="my_collection")



In [ ]:
app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

top_agent = None
# Configure upload folder
UPLOAD_FOLDER = '/content/uploads'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# Ensure the upload folder exists
if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)

agents = {}
query_engines = {}
all_nodes = []

node_parser = SentenceSplitter()


# Function to build agent tools
def build_agent_tools(city_docs):
    """Creates query agents and tools for document interaction."""
    global top_agent
    for wiki_title, doc_nodes in city_docs.items():
        # Split nodes from each document
        nodes = SentenceSplitter().get_nodes_from_documents(doc_nodes)

        # Create indices and query engines
        summary_index = SummaryIndex(nodes)
        vector_index = VectorStoreIndex(nodes)
        vector_query_engine = vector_index.as_query_engine(llm=Settings.llm)
        summary_query_engine = summary_index.as_query_engine(llm=Settings.llm)

        # Define tools for each document
        query_engine_tools = [
            QueryEngineTool(
                query_engine=vector_query_engine,
                metadata=ToolMetadata(name="vector_tool", description=f"Useful for specific queries about {wiki_title}.")
            ),
            QueryEngineTool(
                query_engine=summary_query_engine,
                metadata=ToolMetadata(name="summary_tool", description=f"Provides a holistic summary of {wiki_title}.")
            )
        ]

        # Build agent with the tools
        function_llm = OpenAI(model="gpt-4")
        agent = OpenAIAgent.from_tools(
            query_engine_tools,
            llm=function_llm,
            verbose=True,
            system_prompt=f"Answer queries about {wiki_title}. Use tools provided; do not rely on prior knowledge."
        )

        base_index = VectorStoreIndex(all_nodes)
        base_query_engine = base_index.as_query_engine(similarity_top_k=4)

        agent = OpenAIAgent.from_tools(
        query_engine_tools,
        llm=function_llm,
        verbose=True,
        system_prompt=
          f"""\
          You are a specialized agent designed to answer queries about {wiki_title}.
          You must ALWAYS use at least one of the tools provided when answering a question; do NOT rely on prior knowledge.\
          """,
              )

        top_agent = agent

        # Store agents and query engines for each document
        agents[wiki_title] = agent
        query_engines[wiki_title] = vector_index.as_query_engine(similarity_top_k=2)


# Flask route for document upload
@app.route('/upload', methods=['POST'])
def upload_document():
    try:
        if 'files' not in request.files:
            return jsonify({"error": "No files part in the request."}), 400

        files = request.files.getlist('files')
        if not files:
            return jsonify({"error": "No files uploaded."}), 400


        for file in files:
            filename = secure_filename(file.filename)
            file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            file.save(file_path)  # Save each file to the upload directory

        # Step 2: Define wiki_titles based on uploaded files
        wiki_titles = [secure_filename(file.filename).split('.')[0] for file in files]  # Extract titles from filenames

        # Step 3: Load documents using SimpleDirectoryReader
        city_docs = {}
        for wiki_title in wiki_titles:
            # Load data using SimpleDirectoryReader for each file
            city_docs[wiki_title] = SimpleDirectoryReader(input_files=[file_path]).load_data()

        # Step 4: Initialize collections for agents and nodes
        agents = {}
        query_engines = {}
        all_nodes = []

        # Step 5: Process nodes and upsert embeddings
        for idx, wiki_title in enumerate(wiki_titles):
            nodes = node_parser.get_nodes_from_documents(city_docs[wiki_title])  # Parse nodes from documents
            all_nodes.extend(nodes)

            # Prepare unique IDs for each document
            ids = [f"{wiki_title}_{i}" for i in range(len(nodes))]

            # Upsert each node's text and ID into ChromaDB
            for i, node in enumerate(nodes):
                collection.upsert(
                    documents=[node.text],  # Ensure node.text contains the document text
                    ids=[ids[i]]  # Use unique ID for each document
                )


        build_agent_tools(city_docs)
        return '''
          <!DOCTYPE html>
          <html lang="en">
          <head>
              <meta charset="UTF-8">
              <meta name="viewport" content="width=device-width, initial-scale=1.0">
              <title>Query Interface</title>
          </head>
          <body>
              <h1>Query Interface</h1>
              <p>Upload successful! Please enter your query below.</p>
              <form action="/query" method="POST">
                  <label for="query">Enter your query:</label>
                  <input type="text" id="query" name="query" required>
                  <button type="submit">Submit Query</button>
              </form>
          </body>
          </html>
          '''

    except Exception as e:
        return jsonify({"error": "An error occurred in /upload."}), 500



# Flask route for querying the system
@app.route('/query', methods=['POST'])
def query_system():


    user_query = request.form.get('query')
    print(user_query)
    if not user_query:
        return jsonify({"error": "No query provided."}), 400



    response = top_agent.query(user_query)
    print(response)

    return f'''
      <!DOCTYPE html>
      <html lang="en">
      <head>
          <meta charset="UTF-8">
          <meta name="viewport" content="width=device-width, initial-scale=1.0">
          <title>Query Result</title>
      </head>
      <body>
          <h1>Query Result</h1>
          <p>Query submitted: {user_query}</p>
          <h2>Response:</h2>
          <p>{response}</p>
          <a href="/upload">Upload another document</a>  <!-- Link to go back to upload -->
      </body>
      </html>
      '''

# Flask route for updating a document (PUT)
@app.route('/update_document/<doc_id>', methods=['PUT'])
def update_document(doc_id):
    new_content = request.json.get('content')
    if not new_content:
        return jsonify({"error": "No new content provided."}), 400

    # Delete old embedding, upsert new one
    collection.upsert(documents=[new_content], ids=[doc_id])
    return jsonify({"status": f"Document {doc_id} updated."}), 200

# Flask route for deleting a document (DELETE)
@app.route('/delete_document/<doc_id>', methods=['DELETE'])
def delete_document(doc_id):
    collection.delete(ids=[doc_id])
    return jsonify({"status": f"Document {doc_id} deleted."}), 200

# Flask route for server status check
@app.route('/status', methods=['GET'])
def status():
    return jsonify({"status": "Server is running"}), 200



In [ ]:

@app.route("/", methods=['GET'])
def homepage():

    return '''
    <!doctype html>
    <title>Document Upload and Query System</title>
    <h1>Upload Your Documents</h1>
    <form method="post" action="/upload" enctype="multipart/form-data">
      <input type="file" name="files" multiple>
      <input type="submit" value="Upload">
    </form>


    '''


# Run the Flask app


public_url=ngrok.connect(5000)
print('* ngrok tunnel "{}" -> "http://127.0.0.1:{}"'.format(public_url, 5000))
app.run()

In [ ]:
!pkill ngrok
# Disconnect all open tunnels
for tunnel in ngrok.get_tunnels():
    ngrok.disconnect(tunnel.public_url)

# Kill any remaining ngrok processes
import os
os.system("pkill ngrok")

print("All ngrok tunnels have been reset.")
